In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [ ]:
from dataviztools import bokehutils
from bokeh.plotting import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

In [ ]:
import importlib
importlib.reload(bokehutils)

# Printing stats on rankings

In [ ]:
twinewsRankings = getTwinewsRankings()
models = set()
for row in twinewsRankings.collection.find({"meta.splitVersion": 1, 'meta.maxUsers': None}):
    if row['meta']['model'] != 'combin':
        models.add(row['meta']['model'])
models.remove('worst')
models.remove('ideal')
models.remove('random')
keys = set()
for model in models:
    keys.add(twinewsRankings.collection.find_one({'meta.model': model, "meta.splitVersion": 1, 'meta.maxUsers': None})['id'])
keys.add("combin-8f846")
keys.add("combin-efad4")
print(keys)

In [ ]:
def rankingReport(rankings, maxUsers=None, logger=None, verbose=True, pbarVerbose=True):
    if maxUsers is not None:
        keys = shuffle(list(rankings.keys()), seed=0)[:maxUsers]
        rankings = dictSelect(rankings, keys)
    # We find all data:
    medians = []
    means = []
    rkvs = []
    scoress = []
    allScores = []
    for userId in pb(list(rankings.keys()), logger=logger, verbose=pbarVerbose):
        assert isinstance(rankings[userId][0][0], tuple)
        scores = [e[1] for e in rankings[userId][0]]
        scores = normalizeRankingScores(scores)
        medians.append(np.median(scores))
        means.append(np.mean(scores))
        rkvs.append(rankingVariance(scores))
        scoress.append(scores)
        allScores += list(scores)
    # We print means and std:
    for values, label in \
    [
        (medians, "Median"), (means, "Mean"),
        (rkvs, "Ranking variance")
    ]:
        std = np.std(values)
        # std = mean_confidence_interval(values)
        std = truncateFloat(std, 2)
        mean = truncateFloat(np.mean(values), 2)
        text = label + ": " + str(mean) + " ±" + str(std) + " (95%)"
        log(text, logger, verbose=verbose)
    # We show all scores histogram:
    log("Histogram of all scores (as distances):", logger=logger, verbose=verbose)
    show(bokehutils.barplot(allScores, 30, hideTools=True, hideXAxis=True, hideYAxis=True, title="", height=100, width=300))
    # We show a sample of scores:
    log("Histogram samples of scores (as distances):", logger=logger, verbose=verbose)
    scoress = shuffle(list(scoress))
    figures = []
    for row in range(3):
        currentFigures = []
        for col in range(3):
            scores = scoress.pop()
            f = bokehutils.barplot(scores, 10, hideTools=True, hideXAxis=True, hideYAxis=True, title="", height=40, width=100)
            currentFigures.append(f)
        figures.append(currentFigures)
    grid = gridplot(figures, toolbar_options={'logo': None})
    show(grid)

In [ ]:
tt = TicToc()
tt.tic()
for key in sorted(list(keys)):
    print("#" * 20 + " " + key + " " + "#" * 20)
    rankings = twinewsRankings[key]
    tt.tic("Got " + key + " rankings")
    rankingReport(rankings, maxUsers=None, pbarVerbose=False)
    tt.tic("Stats printed for " + key)
tt.toc()